## Window Theorem (3)

For two separation-identical aircraft $i$ and $j$ where $r_i \le r_j$ and $lt_i \le lt_j$, a *complete-order* can be inferred such that $i$ should precede $j$. Since an aircraft can always be delayed to meet the start of its hard time window, and $t_x \le t'_x$ for all aircraft in corresponding positions of sequences $s$ and $s'$, any time-window violation is no worse in $s$ than in $s'$. Hence, sequencing $i$ before $j$ cannot increase the number or severity of hard time-window violations.


In [12]:
from z3 import *
from itertools import product
from functools import reduce
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from matplotlib import colors
from matplotlib.table import Table


REALISTIC = True



s = Solver()



# introduce some variables
p1, i, p2, j, p3 = Ints('p1 i p2 j p3')
aircraft = [p1, i, p2, j, p3]
s.add(Distinct(*aircraft))

R  = {ac: Real(f"R_{ac}")  for ac in aircraft}
LT = {ac: Real(f"LT_{ac}") for ac in aircraft}
δ   = {(x, y): Real(f"δ_{x}_{y}") for x, y in product(aircraft, aircraft)}



# basic assumptions
for x in aircraft:
    s.add(R[x] > 0, LT[x] > 0)

for (x, y) in product(aircraft, aircraft):
    s.add(δ[x, y] >= 0)
    if x == y:
        s.add(δ[x, y] == 0)



# δ-identical i/j
for x in aircraft:
    s.add(δ[i, x] == δ[j, x])
    s.add(δ[x, i] == δ[x, j])




if REALISTIC:
    for (x, y) in product(aircraft, aircraft):
        s.add(δ[x, y] < 50)
    for (x) in aircraft:
        s.add(R[x] < 100)
        s.add(LT[x] < 100)




# helper (symbolic max)
def zmax(x, y): return If(x >= y, x, y)
def zmax_list(xs): return reduce(lambda a, b: zmax(a, b), xs)

def compute_T(seq):
    T = {seq[0]: R[seq[0]]}
    for k in range(1, len(seq)):
        preds = seq[:k]
        T[seq[k]] = zmax(R[seq[k]],
                         zmax_list([T[p] + δ[p, seq[k]] for p in preds]))
    return T

def window_violations(T):
    return Sum([If(T[a] > LT[a], 1, 0) for a in aircraft])



# encode s and s'
S       = [p1, i, p2, j, p3]
S_prime = [p1, j, p2, i, p3]

T       = compute_T(S)
T_prime = compute_T(S_prime)

WV      = window_violations(T)
WV_p    = window_violations(T_prime)



# theorem assumptions
s.add(R[i] < R[j])
s.add(LT[i] < LT[j])

# S' Is feasible
for ac in aircraft:
    s.add(T_prime[ac] < LT[i])





# hypothesis
s.add(WV > WV_p)



print("Checking counterexample to Theorem 3 (hard time windows)...")
res = s.check()

def z3_to_float(val):
    if is_rational_value(val):
        return val.numerator_as_long() / val.denominator_as_long()
    if is_algebraic_value(val):
        s = val.approx(20)
        if s.endswith("?"): s = s[:-1]
        return float(s)
    s = str(val)
    if s.endswith("?"): s = s[:-1]
    if "/" in s:
        num, den = s.split("/", 1)
        return float(num) / float(den)
    return float(s)

if res == sat:
    print("SAT, Generating Counterexample...")
    m = s.model()
    data = []
    for ac in aircraft:
        r   = z3_to_float(m.evaluate(R[ac], model_completion=True))
        lt  = z3_to_float(m.evaluate(LT[ac], model_completion=True))
        t   = z3_to_float(m.evaluate(T[ac], model_completion=True))
        tp  = z3_to_float(m.evaluate(T_prime[ac], model_completion=True))
        ok_s  = t  <= lt
        ok_sp = tp <= lt
        data.append({
            "Aircraft": str(ac),
            "R": r,
            "LT": lt,
            "T(S)": t,
            "T(S′)": tp,
            "OK(S)": ok_s,
            "OK(S′)": ok_sp,
        })

    df = pd.DataFrame(data)

    delta_matrix = np.array([
        [z3_to_float(m.evaluate(δ[a, b], model_completion=True)) for b in aircraft]
        for a in aircraft
    ])
    labels = [str(a) for a in aircraft]

    fig, (ax_heat, ax_table) = plt.subplots(1, 2, figsize=(10, 4),
                                            gridspec_kw={'width_ratios': [1, 1.4]})
    plt.subplots_adjust(wspace=0.4)

    im = ax_heat.imshow(delta_matrix, cmap="viridis", interpolation="nearest")

    ax_heat.set_title("Separation Matrix δ(x, y)")
    ax_heat.set_xticks(np.arange(len(labels)))
    ax_heat.set_yticks(np.arange(len(labels)))
    ax_heat.set_xticklabels(labels)
    ax_heat.set_yticklabels(labels)
    plt.setp(ax_heat.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor")

    # Draw numeric values on top of heatmap
    for i in range(len(labels)):
        for j in range(len(labels)):
            val = delta_matrix[i, j]
            text_color = "white" if val < np.max(delta_matrix) / 2 else "black"
            ax_heat.text(j, i, f"{val:.2f}",
                         ha="center", va="center", color=text_color, fontsize=9, fontweight='bold')

    fig.colorbar(im, ax=ax_heat, fraction=0.046, pad=0.04)

    ax_table.axis("off")
    tbl = Table(ax_table, bbox=[0, 0, 1, 1])

    n_rows, n_cols = len(df), 5
    col_labels = ["Aircraft", "R", "LT", "T(S)", "T(S′)"]
    col_widths = [0.2, 0.15, 0.15, 0.25, 0.25]
    row_height = 1.0 / (n_rows + 1)

    for j, label in enumerate(col_labels):
        cell = tbl.add_cell(0, j, width=col_widths[j], height=row_height,
                            text=label, loc='center', facecolor='#40466e')
        cell.get_text().set_color('w')
        cell.get_text().set_weight('bold')

    for i, row in enumerate(df.itertuples(), start=1):
        vals = [row.Aircraft, f"{row.R:.3f}", f"{row.LT:.3f}",
                f"{row._4:.3f}", f"{row._5:.3f}"]
        for j, val in enumerate(vals):
            facecolor = 'white'
            if col_labels[j] == "T(S)":
                facecolor = '#c8e6c9' if row._6 else '#ffcdd2'   # green/red
            elif col_labels[j] == "T(S′)":
                facecolor = '#c8e6c9' if row._7 else '#ffcdd2'
            cell = tbl.add_cell(i, j, width=col_widths[j], height=row_height,
                                text=val, loc='center', facecolor=facecolor)
            cell.get_text().set_color('black')

    ax_table.add_table(tbl)
    ax_table.set_title("Timing and Window Compliance", fontsize=12, pad=12)

    plt.show()

else:
    print("\nUNSAT — Theorem 3 holds: a before b cannot worsen hard-window compliance if s' is feasible.")


Checking counterexample to Theorem 3 (hard time windows)...

UNSAT — Theorem 3 holds: a before b cannot worsen hard-window compliance if s' is feasible.
